In [44]:
# import itertools as it
# from multiprocessing import Pool, cpu_count
# from collections import defaultdict
import re
from math import gcd

with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279
"""


### Part 1
Ok. A math problem.

Given `a,b,n, ax+bx=n`: what is `a` and `b`?
- If `n,a,b` in dict, return value
- Use [Linear Diophantine Equation](https://www.geeksforgeeks.org/linear-diophantine-equations/) to find out if a solution exist
- if solution exist: Check `(n-i*a) % b == 0` for `i<100`


In [56]:
class ClawMachine():
    def __init__ (self,txt,part2=False):
        pattern = r"Button A: X\+(\d+), Y\+(\d+)\nButton B: X\+(\d+), Y\+(\d+)\nPrize: X=(\d+), Y=(\d+)"
        matches = re.findall(pattern,txt)

        self.solutions = []
        self.cost = 0
        for match in matches:
            aX, aY, bX, bY, prizeX, prizeY = map(int, match) # mapping each value in match to int, assigning to var
            if (part2):
                prizeX += 10000000000000
                prizeY += 10000000000000
            resX = self.find_solutions(prizeX,aX,bX)
            resY = self.find_solutions(prizeY,aY,bY)
            common = set(resX).intersection(resY)
            if common:
                resA,resB = min(common,key=lambda i: i[0]*3+i[1])
                self.cost += resA*3 + resB

    # find solution
    def find_solutions(self,n,a,b):
        # n,a,b = info
        # print("start of sols")
        sols = []
        gcdAB = gcd(a,b)
        if n % gcdAB != 0: return sols
        for i in range(100):
            if (n-(i*a))%b == 0:
                sols.append((i,int(n-(i*a))/b))
        # print("for a:",a,", b:",b,", n:",n,"=",sols)
        # print(sols)
        return set(sols)


p1 = ClawMachine(test_input)
# p2 = ClawMachine(test_input,True)
print("Part 1:",int(p1.cost))
# print("Part 2:",p2.cost)

Part 1: 480


Some extra tests I've done

In [13]:
# Initialization of list
list1 = [(6, 2), (3, 4), (1,6)]
list2 = [(2, 3), (90, 22),(6,2),(1,6)]
#Finding intersection
common_elements = set(list1).intersection(list2)
print(min(common_elements,key=lambda i: i[0]*3+i[1]))

(1, 6)


### Part 2
So this was once again an efficiency problem. Cursory search led me to Cramer's rule, and now I'm remembering all about it (lie). I had to re-learn all this (using this [Wraith of Math video](https://www.youtube.com/watch?v=ZbXOuukX6cA) with a 2x2 example) and apply it, and once I figured out the function it was smooth sailing.

In [58]:
def solve(aX, aY, bX, bY, prizeX, prizeY):
    # x  y
    # aX bX = prizeX
    # aY bY = prizeY
    detA = calc_det(aX,bX,aY,bY)
    Dx = calc_det(prizeX,bX,prizeY,bY)
    Dy = calc_det(aX,prizeX,aY,prizeY)
    return (Dx/detA,Dy/detA)

def calc_det(a,b,c,d):
    #a b
    #c d
    return (a*d)-(c*b)

solve(26,66,67,21,10000000012748,10000000012176) # example 2

(118679050709.0, 103199174542.0)

And then it's just using the new solve to do it!

### Cleaned up + Final solution:

In [69]:
class ClawMachine():
    def __init__ (self,txt,part2=False):
        pattern = r"Button A: X\+(\d+), Y\+(\d+)\nButton B: X\+(\d+), Y\+(\d+)\nPrize: X=(\d+), Y=(\d+)"
        matches = re.findall(pattern,txt)

        self.solutions = []
        self.cost = 0
        for match in matches:
            aX, aY, bX, bY, prizeX, prizeY = map(int, match) # mapping each value in match to int, assigning to var
            if (part2):
                prizeX += 10000000000000
                prizeY += 10000000000000
            solution = self.solve(aX, aY, bX, bY, prizeX, prizeY)
            # print(solution)
            self.cost += solution[0]*3+solution[1]

    def solve(self,aX, aY, bX, bY, prizeX, prizeY):
        # x  y
        # aX bX = prizeX
        # aY bY = prizeY
        detA = calc_det(aX,bX,aY,bY)
        Dx = calc_det(prizeX,bX,prizeY,bY)
        Dy = calc_det(aX,prizeX,aY,prizeY)
        if Dx%detA == 0 and Dy%detA == 0:
            return (int(Dx/detA),int(Dy/detA))
        else:
            return (0,0)

    def calc_det(a,b,c,d):
        #a b
        #c d
        return (a*d)-(c*b)


p1 = ClawMachine(input)
p2 = ClawMachine(input,True)
print("Part 1:",p1.cost)
print("Part 2:",p2.cost)

Part 1: 29438
Part 2: 104958599303720


### Takeaways
- math... sometimes it's the intuition of knowing to use linear algebra when it comes down to it.
- I kept thinking that there would be more than one solution for each equation, a simple plotting would've helped me figure out they're linear equations and would only have one answer. I did learn some tricks for intersections though, overall pretty fun day.